In [1]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from category_encoders import TargetEncoder
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy.stats.mstats import winsorize
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import optuna
import warnings
warnings.filterwarnings('ignore')

/home/huy/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Handle outlier

In [2]:
class OutlierHandler:
    """
    Handles outlier detection and removal using multiple methods:
    - Winsorization for heavy skewed features
    - Hybrid approach: IsolationForest + Mahalanobis distance
    """
    
    def __init__(self, random_state=42):
        self.random_state = random_state
        self.report = {}

    def detect_heavy_skew(self, df, numeric_cols, threshold=1.0):
        """Detect columns with high skewness"""
        valid_cols = [col for col in numeric_cols if col in df.columns]
        if not valid_cols:
            return [], {}

        skewed = df[valid_cols].skew().abs().sort_values(ascending=False)
        heavy_skew = skewed[skewed > threshold].index.tolist()

        return heavy_skew, skewed[skewed > threshold].to_dict()

    def winsorize_columns(self, df, columns, lower_pct=0.01, upper_pct=0.99):
        """Apply winsorization to specified columns"""
        df_wins = df.copy()

        for col in columns:
            if col not in df_wins.columns:
                continue
            arr = df_wins[col].astype(float)
            low_val, high_val = arr.quantile(lower_pct), arr.quantile(upper_pct)
            df_wins[col] = arr.clip(lower=low_val, upper=high_val)

        return df_wins

    def hybrid_outlier_detection(self, df, numeric_cols, contamination='auto'):
        """
        Detect outliers using both IsolationForest and Mahalanobis distance
        Only removes rows flagged by both methods (conservative approach)
        """
        valid_cols = [col for col in numeric_cols if col in df.columns]
        if not valid_cols or len(df) < 100:
            return np.ones(len(df), dtype=bool), {}

        X = df[valid_cols].copy()

        # Auto contamination
        if contamination == 'auto':
            iqr_outliers = []
            for col in valid_cols:
                Q1, Q3 = df[col].quantile([0.25, 0.75])
                IQR = Q3 - Q1
                iqr_outliers.append(((df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)).sum())
            contamination = min(0.05, max(0.01, np.mean(iqr_outliers) / len(df)))

        # Impute and detect
        Xf = SimpleImputer(strategy='median').fit_transform(X)

        # IsolationForest
        iso = IsolationForest(contamination=contamination, random_state=self.random_state, n_estimators=100)
        iso_mask = iso.fit_predict(Xf) == -1

        # PCA + Mahalanobis
        try:
            Xs = StandardScaler().fit_transform(Xf)
            pca = PCA(n_components=0.90, random_state=self.random_state)
            pcs = pca.fit_transform(Xs)

            cov = np.cov(pcs, rowvar=False)
            inv_cov = np.linalg.pinv(cov)
            mean_vec = pcs.mean(axis=0)
            dists = np.sqrt(np.sum((pcs - mean_vec) @ inv_cov * (pcs - mean_vec), axis=1))

            mahal_mask = dists > np.percentile(dists, 100*(1-contamination))
        except:
            mahal_mask = np.zeros(len(X), dtype=bool)

        # Conservative: remove only if both agree
        outlier_mask = iso_mask & mahal_mask
        keep_mask = ~outlier_mask

        print(f"Outliers detected: IsoForest={iso_mask.sum()}, Mahalanobis={mahal_mask.sum()}, Both={outlier_mask.sum()}")
        print(f"Keep: {keep_mask.sum()}/{len(df)} ({100*keep_mask.mean():.1f}%)")

        return keep_mask, {'consensus_outliers': outlier_mask.sum(), 'contamination': contamination}

    def process_outliers(self, df, numeric_cols, method='both', skew_threshold=1.0,
                        contamination='auto', action='remove'):
        """Main outlier processing pipeline"""
        print(f"\n{'='*60}\nOUTLIER PROCESSING: {method} | {action}")
        print(f"Original: {df.shape}")

        df_clean = df.copy()

        # 1. Detect & Winsorize heavy skew
        if method in ['winsorize_only', 'both']:
            heavy_skew_cols, _ = self.detect_heavy_skew(df_clean, numeric_cols, skew_threshold)
            if heavy_skew_cols:
                df_clean = self.winsorize_columns(df_clean, heavy_skew_cols)

        # 2. Hybrid outlier detection
        if method in ['hybrid', 'both']:
            keep_mask, details = self.hybrid_outlier_detection(df_clean, numeric_cols, contamination)

            if action == 'remove':
                df_clean = df_clean[keep_mask].reset_index(drop=True)
                print(f"Removed {(~keep_mask).sum()} rows")

        print(f"Final: {df_clean.shape} | Removed: {len(df)-len(df_clean)} rows ({100*(len(df)-len(df_clean))/len(df):.1f}%)")
        print("="*60)

        return df_clean, self.report

# DATA PREPROCESSING

In [3]:
def load_and_clean_data(filepath):
    """Load data and drop unnecessary columns"""
    house_df = pd.read_csv(filepath)
    house_df = house_df.drop(["Id", "Alley", "PoolQC", "Fence", "MiscFeature"], axis=1)
    return house_df


def prepare_features(train_df, test_df, use_polynomial=False, poly_degree=2, top_n_features=10):
    
    # Extract target
    y_train = train_df["SalePrice"].values
    y_test = test_df["SalePrice"].values
    x_train = train_df.drop(["SalePrice"], axis=1)
    x_test = test_df.drop(["SalePrice"], axis=1)

    # Identify column types
    num_cols = [col for col in x_train.columns if x_train[col].dtype in ["float64", "int64"]]
    cat_cols = [col for col in x_train.columns if x_train[col].dtype not in ["float64", "int64"]]

    # Fill missing values for categorical columns
    x_train[cat_cols] = x_train[cat_cols].fillna("none")
    x_test[cat_cols] = x_test[cat_cols].fillna("none")

    # Target encoding for categorical variables
    target_encoder = TargetEncoder(
        cols=cat_cols,
        smoothing=1.0,
        min_samples_leaf=1,
        handle_unknown='value',
        handle_missing='value'
    )
    X_train_encoded = target_encoder.fit_transform(x_train, y_train)
    X_test_encoded = target_encoder.transform(x_test)

    # Impute numerical values
    imputer = SimpleImputer()
    X_train_encoded[num_cols] = imputer.fit_transform(x_train[num_cols])
    X_test_encoded[num_cols] = imputer.transform(x_test[num_cols])

    # Apply polynomial features if requested
    if use_polynomial:
        print(f"\n{'='*60}")
        print(f"APPLYING POLYNOMIAL FEATURES (degree={poly_degree})")
        print(f"{'='*60}")
        
        # Select top correlated features for polynomial transformation
        correlations = x_train[num_cols].corrwith(pd.Series(y_train, index=x_train.index)).abs()
        top_features = correlations.nlargest(top_n_features).index.tolist()
        print(f"Top {top_n_features} features selected for polynomial transformation:")
        print(top_features)
        
        # Apply polynomial features to selected numerical features
        poly = PolynomialFeatures(degree=poly_degree, include_bias=False)
        X_train_poly = poly.fit_transform(x_train[top_features])
        X_test_poly = poly.transform(x_test[top_features])
        
        # Combine with categorical features
        X_train_cat = X_train_encoded[cat_cols].values
        X_test_cat = X_test_encoded[cat_cols].values
        
        X_train_final = np.hstack([X_train_poly, X_train_cat])
        X_test_final = np.hstack([X_test_poly, X_test_cat])
        
        print(f"Shape after polynomial features: {X_train_final.shape}")
    else:
        X_train_final = X_train_encoded.values
        X_test_final = X_test_encoded.values

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_final)
    X_test_scaled = scaler.transform(X_test_final)

    return X_train_scaled, X_test_scaled, y_train, y_test

# MODEL OPTIMIZATION

In [4]:
def get_model_configs():
    return {
        'Ridge': {
            'model_class': Ridge,
            'params': lambda trial: {
                'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True)
            }
        },
        'Lasso': {
            'model_class': Lasso,
            'params': lambda trial: {
                'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True)
            }
        },
        'ElasticNet': {
            'model_class': ElasticNet,
            'params': lambda trial: {
                'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
                'l1_ratio': trial.suggest_float('l1_ratio', 0.0, 1.0)
            }
        },
        'RandomForest': {
            'model_class': RandomForestRegressor,
            'params': lambda trial: {
                'n_estimators': trial.suggest_int('n_estimators', 100, 600),
                'max_depth': trial.suggest_int('max_depth', 3, 30),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', 1.0]),
                'random_state': 42,
                'n_jobs': -1
            }
        },
        'GradientBoosting': {
            'model_class': GradientBoostingRegressor,
            'params': lambda trial: {
                'n_estimators': trial.suggest_int('n_estimators', 100, 500),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
                'max_depth': trial.suggest_int('max_depth', 2, 8),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'random_state': 42
            }
        },
        'XGBoost': {
            'model_class': XGBRegressor,
            'params': lambda trial: {
                'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
                'max_depth': trial.suggest_int('max_depth', 3, 12),
                'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10, log=True),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10, log=True),
                'random_state': 42,
                'n_jobs': -1,
                'tree_method': "hist"
            }
        },
        'LightGBM': {
            'model_class': LGBMRegressor,
            'params': lambda trial: {
                'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
                'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.2, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 15, 127),
                'max_depth': trial.suggest_int('max_depth', -1, 10),
                'min_child_samples': trial.suggest_int('min_child_samples', 5, 30),
                'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
                'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10, log=True),
                'random_state': 42,
                'n_jobs': -1,
                'verbose': -1
            }
        },
        'CatBoost': {
            'model_class': CatBoostRegressor,
            'params': lambda trial: {
                'iterations': trial.suggest_int('iterations', 300, 1000),
                'depth': trial.suggest_int('depth', 4, 10),
                'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10, log=True),
                'random_seed': 42,
                'verbose': False
            }
        }
    }


def objective(trial, model_name, model_config, X_train, y_train, X_test, y_test):
    """Objective function for Optuna optimization"""
    # Get model parameters
    params = model_config['params'](trial)
    
    # Create and train model
    model = model_config['model_class'](**params)
    model.fit(X_train, y_train)
    
    # Predict and evaluate
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    # Store RMSE as user attribute
    trial.set_user_attr("rmse", rmse)
    
    return r2  # Optuna will maximize this


def optimize_models(X_train, y_train, X_test, y_test, model_names=None, n_trials=30):
    
    model_configs = get_model_configs()
    
    if model_names is None:
        model_names = list(model_configs.keys())
    
    best_results = {}
    
    for model_name in model_names:
        if model_name not in model_configs:
            print(f"Warning: {model_name} not found in model configs. Skipping.")
            continue
            
        print(f"\n{'='*60}")
        print(f"Optimizing {model_name}...")
        print(f"{'='*60}")
        
        model_config = model_configs[model_name]
        
        # Create study
        study = optuna.create_study(direction="maximize")
        study.optimize(
            lambda trial: objective(trial, model_name, model_config, X_train, y_train, X_test, y_test),
            n_trials=n_trials,
            show_progress_bar=True
        )
        
        # Store best results
        best_results[model_name] = {
            "best_r2": study.best_value,
            "best_rmse": study.best_trial.user_attrs["rmse"],
            "best_params": study.best_params
        }
    
    return best_results


def print_results(results, title="RESULTS"):
    print(f"{title}")
    
    results_df = pd.DataFrame(results).T
    results_df = results_df.sort_values("best_r2", ascending=False)
    print(results_df)
    
    print("DETAILED RESULTS")
    
    for model_name, result in results.items():
        print(f"\n{model_name}:")
        print(f"   R²: {result['best_r2']:.4f}")
        print(f"   RMSE: {result['best_rmse']:.2f}")
        print(f"   Parameters:")
        for param, value in result['best_params'].items():
            if isinstance(value, float):
                print(f"      {param}: {value:.6f}")
            else:
                print(f"      {param}: {value}")

# COMPARISON 

In [5]:

def compare_models_with_and_without_poly(
    df_clean,
    models_to_test=None,
    n_trials=30,
    poly_degree=2,
    top_n_features=10
):
    # Split data
    train_df, test_df = train_test_split(df_clean, test_size=0.25, random_state=42)
    
    # WITHOUT POLYNOMIAL FEATURES 
    print("\n" + "="*70)
    print("STAGE 1: STANDARD FEATURES (No Polynomial)")
    print("="*70)
    
    X_train_std, X_test_std, y_train, y_test = prepare_features(
        train_df, test_df, use_polynomial=False
    )
    
    results_standard = optimize_models(
        X_train_std, y_train, X_test_std, y_test,
        model_names=models_to_test,
        n_trials=n_trials
    )
    
    print_results(results_standard, "RESULTS - STANDARD FEATURES")
    
    #  WITH POLYNOMIAL FEATURES 
    print("\n" + "="*70)
    print("STAGE 2: POLYNOMIAL FEATURES")
    print("="*70)
    
    X_train_poly, X_test_poly, y_train, y_test = prepare_features(
        train_df, test_df, 
        use_polynomial=True,
        poly_degree=poly_degree,
        top_n_features=top_n_features
    )
    
    results_poly = optimize_models(
        X_train_poly, y_train, X_test_poly, y_test,
        model_names=models_to_test,
        n_trials=n_trials
    )
    
    print_results(results_poly, "RESULTS - POLYNOMIAL FEATURES")
    
    #  COMPARISON 
    print("FINAL COMPARISON: STANDARD vs POLYNOMIAL")
    
    comparison_data = []
    for model_name in results_standard.keys():
        comparison_data.append({
            'Model': model_name,
            'Standard_R2': results_standard[model_name]['best_r2'],
            'Standard_RMSE': results_standard[model_name]['best_rmse'],
            'Poly_R2': results_poly[model_name]['best_r2'],
            'Poly_RMSE': results_poly[model_name]['best_rmse'],
            'R2_Improvement': results_poly[model_name]['best_r2'] - results_standard[model_name]['best_r2'],
            'RMSE_Improvement': results_standard[model_name]['best_rmse'] - results_poly[model_name]['best_rmse']
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    comparison_df = comparison_df.sort_values('Poly_R2', ascending=False)
    
    print("\n", comparison_df.to_string(index=False))
    
    print("SUMMARY")
    print(f"Models improved with polynomial features: {(comparison_df['R2_Improvement'] > 0).sum()}/{len(comparison_df)}")
    print(f"Best model (Standard): {comparison_df.loc[comparison_df['Standard_R2'].idxmax(), 'Model']}")
    print(f"Best model (Polynomial): {comparison_df.loc[comparison_df['Poly_R2'].idxmax(), 'Model']}")
    
    return results_standard, results_poly, comparison_df



In [6]:
# MAIN EXECUTION
def main(
    filepath,
    models_to_test=None,
    n_trials=30,
    poly_degree=2,
    top_n_features=10
):
    """
    Main execution function
    
    Parameters:
    -----------
    filepath : str
        Path to the dataset
    models_to_test : list or None
        List of model names to test. Available models:
        ['Ridge', 'Lasso', 'ElasticNet', 'RandomForest', 'GradientBoosting', 
         'XGBoost', 'LightGBM', 'CatBoost']
        If None, test all models.
    n_trials : int
        Number of optimization trials per model
    poly_degree : int
        Degree of polynomial features
    top_n_features : int
        Number of top features to apply polynomial transformation
    """
    print("="*70)
    print("UNIFIED MODEL COMPARISON: STANDARD vs POLYNOMIAL FEATURES")
    print("="*70)
    
    # Load and clean data
    print("\n[1/4] Loading and cleaning data...")
    house_df = load_and_clean_data(filepath)
    print(f"Data loaded: {house_df.shape}")
    
    # Handle outliers
    print("\n[2/4] Processing outliers...")
    num_cols_org = [col for col in house_df.columns if house_df[col].dtype in ["float64", "int64"]]
    handler = OutlierHandler(random_state=42)
    df_clean, report = handler.process_outliers(
        house_df,
        num_cols_org,
        method='both',
        skew_threshold=1.0,
        contamination='auto',
        action='remove'
    )
    
    # Compare models
    print("\n[3/4] Optimizing models...")
    results_standard, results_poly, comparison_df = compare_models_with_and_without_poly(
        df_clean,
        models_to_test=models_to_test,
        n_trials=n_trials,
        poly_degree=poly_degree,
        top_n_features=top_n_features
    )
    
    print("\n[4/4] Done!")
    
    return results_standard, results_poly, comparison_df


In [9]:
results_std, results_poly, comparison = main(filepath="/home/huy/quan_nguyen/module 5/project/train-house-prices-advanced-regression-techniques.csv", 
    n_trials=10)

UNIFIED MODEL COMPARISON: STANDARD vs POLYNOMIAL FEATURES

[1/4] Loading and cleaning data...
Data loaded: (1460, 76)

[2/4] Processing outliers...

OUTLIER PROCESSING: both | remove
Original: (1460, 76)
Outliers detected: IsoForest=45, Mahalanobis=45, Both=13
Keep: 1447/1460 (99.1%)
Removed 13 rows
Final: (1447, 76) | Removed: 13 rows (0.9%)

[3/4] Optimizing models...

STAGE 1: STANDARD FEATURES (No Polynomial)


[I 2025-11-01 20:49:25,525] A new study created in memory with name: no-name-0d692bba-963c-42e8-9fef-449fb5bbd5eb



Optimizing Ridge...


Best trial: 3. Best value: 0.894286: 100%|██████████| 10/10 [00:00<00:00, 312.99it/s]
[I 2025-11-01 20:49:25,559] A new study created in memory with name: no-name-fbd2e064-89e9-4f73-9b5c-ec245889c72c


[I 2025-11-01 20:49:25,528] Trial 0 finished with value: 0.8930085828972408 and parameters: {'alpha': 0.00459199414868515}. Best is trial 0 with value: 0.8930085828972408.
[I 2025-11-01 20:49:25,531] Trial 1 finished with value: 0.8930043978204523 and parameters: {'alpha': 0.0019340147881435157}. Best is trial 0 with value: 0.8930085828972408.
[I 2025-11-01 20:49:25,534] Trial 2 finished with value: 0.8930287108671465 and parameters: {'alpha': 0.017456372648248803}. Best is trial 2 with value: 0.8930287108671465.
[I 2025-11-01 20:49:25,537] Trial 3 finished with value: 0.8942861333349107 and parameters: {'alpha': 1.2398980042060979}. Best is trial 3 with value: 0.8942861333349107.
[I 2025-11-01 20:49:25,541] Trial 4 finished with value: 0.8933535311152285 and parameters: {'alpha': 0.24563064418057737}. Best is trial 3 with value: 0.8942861333349107.
[I 2025-11-01 20:49:25,544] Trial 5 finished with value: 0.8930036923060392 and parameters: {'alpha': 0.0014865028621929967}. Best is tria

...

[I 2025-11-01 20:52:11,433] Trial 7 finished with value: 0.9142879163588865 and parameters: {'iterations': 720, 'depth': 4, 'learning_rate': 0.18218168094746442, 'l2_leaf_reg': 0.010634222465154596}. Best is trial 1 with value: 0.915993136182695.


Best trial: 1. Best value: 0.915993:  90%|█████████ | 9/10 [00:41<00:03,  3.61s/it]

[I 2025-11-01 20:52:12,005] Trial 8 finished with value: 0.9139636739071701 and parameters: {'iterations': 745, 'depth': 5, 'learning_rate': 0.0724990688731641, 'l2_leaf_reg': 0.0400813727037115}. Best is trial 1 with value: 0.915993136182695.


Best trial: 1. Best value: 0.915993: 100%|██████████| 10/10 [00:41<00:00,  4.19s/it]

[I 2025-11-01 20:52:12,457] Trial 9 finished with value: 0.4417903403549983 and parameters: {'iterations': 390, 'depth': 6, 'learning_rate': 0.00118196537836435, 'l2_leaf_reg': 3.285121776352345}. Best is trial 1 with value: 0.915993136182695.
RESULTS - POLYNOMIAL FEATURES
                   best_r2     best_rmse  \
XGBoost           0.916509  20261.989706   
CatBoost          0.915993  20324.432989   
GradientBoosting  0.915034  20440.108309   
LightGBM          0.912131  20786.426945   
RandomForest      0.906155  21481.630027   
Ridge             0.905707  21532.764711   
ElasticNet        0.905361  21572.337607   
Lasso             0.899478   22232.71783   

                                                        best_params  
XGBoost           {'n_estimators': 948, 'max_depth': 3, 'learnin...  
CatBoost          {'iterations': 412, 'depth': 10, 'learning_rat...  
GradientBoosting  {'n_estimators': 490, 'learning_rate': 0.05683...  
LightGBM          {'n_estimators': 295, 'learning